In [268]:
from __future__ import print_function
from ortools.linear_solver import pywraplp
import random
import numpy as np
from random import randint
    
def exact_solution(matrix = [], N = 1):
    solver = pywraplp.Solver.CreateSolver('CLP')
    L = len(matrix[0])
    M = len(matrix)
    a = []
    for i in range(M):
        current = []
        for j in range(L):
            current.append(solver.BoolVar('a' + str(i) + str(j)))
        a.append(current)
    y = []
    for i in range(L):
        y.append(solver.BoolVar('y' + str(i)))
    
                 
    main_con = solver.Constraint(N, N)
    for i in range(L):
        main_con.SetCoefficient(y[i], 1)
    
    cons = []
    for i in range(M):
        cons.append(solver.Constraint(1, 1))
        for j in range(L):
            cons[i].SetCoefficient(a[i][j],1)
    
    sec_cons = []
    for i in range (M):
        s_cons = []
        for j in range (L):
            s_cons.append(solver.Constraint(-1, 0))
            s_cons[j].SetCoefficient(y[j], -1)
            s_cons[j].SetCoefficient(a[i][j], 1)
            
    
    objective = solver.Objective()
    for i in range(M):
        for j in range(L):
            objective.SetCoefficient(a[i][j], matrix[i][j])
    objective.SetMinimization()
    solver.Solve()
    solution = []
    for i in range (L):
        solution.append(y[i].solution_value())
    return solution, objective.Value()
    

def distance (c1 = [], c2 = []):
    result = 0
    for i in range(len(c1)):
        result = result + np.power(c1[i] - c2[i],2)
    return result
    
    
def create_distance_matrix(n = 3, k = 3):
    coordinates = []
    possible_locations = []
    for i in range(n):
        current = []
        current.append(100 * random.random() - 50)
        current.append(100 * random.random() - 50)
        coordinates.append(current)
    for i in range(k):
        current = []
        current.append(100 * random.random() - 50)
        current.append(100 * random.random() - 50)
        possible_locations.append(current)
    matrix = []
    for i in range(len(coordinates)):
        row = []
        for j in range(len(possible_locations)):
            row.append(distance(c1 = coordinates[i], c2 = possible_locations[j]))
        matrix.append(row)
    return matrix

In [330]:
def threshold_approach(matrix, N, threshold, penalty):
    solver = pywraplp.Solver.CreateSolver('CLP')
    L = len(matrix[0])
    M = len(matrix)
    a = []
    indexes = []
    for i in range(M):
        current = []
        current_indexes = []
        for j in range(L):
            if (matrix[i][j] < threshold):
                current.append(solver.BoolVar('a' + str(i) + str(j)))
                current_indexes.append(j)
        a.append(current)
        indexes.append(current_indexes)
        
    y = []
    for i in range(L):
        y.append(solver.BoolVar('y' + str(i)))
        
    f = []
    for i in range(M):
        f.append(solver.BoolVar('f' + str(i)))
        
    main_con = solver.Constraint(N, N)
    for i in range(L):
        main_con.SetCoefficient(y[i], 1)    
        
    cons = []
    for i in range(M):
        cons.append(solver.Constraint(1, 1))
        for j in range(len(indexes[i])):
            cons[i].SetCoefficient(a[i][j],1)
        cons[i].SetCoefficient(f[i],1)
    
    sec_cons = []
    for i in range (M):
        s_cons = []
        for j in range (len(indexes[i])):
            s_cons.append(solver.Constraint(-1, 0))
            s_cons[j].SetCoefficient(y[indexes[i][j]], -1)
            s_cons[j].SetCoefficient(a[i][j], 1)
    
    
    objective = solver.Objective()
    for i in range(M):
        for j in range(len(indexes[i])):
            objective.SetCoefficient(a[i][j], matrix[i][indexes[i][j]])
        objective.SetCoefficient(f[i], penalty)
    objective.SetMinimization()
    solver.Solve()
    solution = []
    for i in range (L):
        solution.append(y[i].solution_value())
    return solution, objective.Value()


def Threshold_Exact(matrix, N):
    stop = False
    Threshold = 200
    while(stop == False):
        LowPenalty, LowPenalty_value = threshold_approach(matrix, N, Threshold, Threshold)
        HighPenalty, High_penalty_value = threshold_approach(matrix, N, Threshold, 1000 * Threshold)
        if (LowPenalty_value != High_penalty_value):
            Threshold = Threshold + 200
        else:
            stop = True
            return LowPenalty, LowPenalty_value
    

In [331]:
for i in range(100):
    demand_points = randint(1, 50)
    possible_locations = randint(1, 100)
    AEDs = randint(1, demand_points)
    matrix = create_distance_matrix(n = demand_points, k=possible_locations)
    thr, thr_obj = Threshold_Exact(matrix, AEDs)
    exact, exact_obj = exact_solution(matrix, AEDs)
    print(exact_obj + " " + thr_obj)

9120.08155749268
___________________
1709.5867905598052
___________________
0.0
___________________
3585.4391992170326
___________________
0.0
___________________
772.7596569021155
___________________
2228.3932818693024
___________________
5493.388574722619
___________________
832.9260389639969
___________________
0.0
___________________
6239.289915248495
___________________
0.0
___________________
2866.7529947852236
___________________
617.4579096565129
___________________
614.078333967686
___________________
3398.6027248227633
___________________
0.0
___________________
5.439500725903931
___________________
14009.524080785332
___________________
68518.12087705758
___________________
1501.2461748609214
___________________
42400.858744630306
___________________
5981.441670938897
___________________
2827.0137989347645
___________________
3274.164365741381
___________________
3700.933027224054
___________________
2954.75608292566
___________________
87850.24742767813
___________________
